<a href="https://colab.research.google.com/github/maramatef111/Elevvo-Internship-Tasks/blob/main/Traffic_Sign_Recognition_Description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout ,Conv2D,MaxPool2D,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import Image
import csv
import cv2

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [ ]:
PATH = '/content/drive/MyDrive/Traffic_Sign_Recognition _Description'
images = [] # images
labels = [] # corresponding labels

# loop over all 43 classes
gtFile = open(PATH + '/Train.csv') # annotations file
gtReader = csv.reader(gtFile, delimiter=',') # csv parser for annotations file
next(gtReader) # skip header
# loop over all images in current annotations file
for row in gtReader:
    img = cv2.imread(PATH + '/' + row[7])
    if img is not None:
        images.append(cv2.resize(img, (28, 28)))
        labels.append(row[6]) # the 6th column is the label
gtFile.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print('Number of loaded images: ' + str(len(images)))
print('Number of loaded labels: ' + str(len(labels)))

Number of loaded images: 5259
Number of loaded labels: 5259


In [ ]:
images = np.asarray(images)
images = images / 255
images = np.asarray(images, dtype = "float32")
labels = np.asarray(labels, dtype= "int32")

In [ ]:
print('Shape of training array: ' + str(images.shape))

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x=labels,palette='viridis')
plt.title('The Count of images in training data')
plt.xlabel('labels')
plt.ylabel('count')
plt.show()

In [ ]:
def preview(images, labels):
    plt.figure(figsize=(16, 16))
    for c in range(len(np.unique(labels))):
        i = np.random.choice(np.where(labels == c)[0])
        plt.subplot(10, 10, c+1)
        plt.axis('off')
        plt.title('class: {}'.format(c))
        plt.imshow(images[i])
preview(images, labels)

In [ ]:
# New desired size
new_size = (32, 32)

# Resize all images
images = np.array([cv2.resize(img, new_size) for img in images])

print('Shape of resized array:', images.shape)

In [ ]:
labels = np.array([0, 1, 0, 2, 1])
np.where(labels == 1)
# Output: (array([1, 4]),)  ← tuple with one array

print(result)       # (array([1, 4]),)
print(result[0])    # array([1, 4])

In [ ]:
def preview(images, labels):
    # Create a new figure with a specified size (16 inches by 16 inches)
    plt.figure(figsize=(16, 16))

    # Loop through each unique class label
    for c in range(len(np.unique(labels))):
        # Randomly choose one index of an image belonging to the current class 'c'
        i = np.random.choice(np.where(labels == c)[0])

        # Create a subplot (10 rows × 10 columns), placing the image in position (c+1)
        plt.subplot(10, 10, c + 1)

        # Hide the axis (no ticks, no border)
        plt.axis('off')
         # Set the title of this subplot to show the class number
        plt.title('class: {}'.format(c))

        # Display the selected image
        plt.imshow(images[i])

# Call the function to preview one sample image from each class
preview(images, labels)


Split Data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    images, labels,
    test_size=0.2,   # 20% for validation
    stratify=labels,  # keep class balance
    random_state=42
)

print("Train:", X_train.shape, "Validation:", X_val.shape)

Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest"
)

# Validation data (no augmentation, no rescale)
val_datagen = ImageDataGenerator()

from tensorflow.keras.utils import to_categorical

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=43)
y_val_cat = to_categorical(y_val, num_classes=43)

# Now flow with the encoded labels
train_generator = train_datagen.flow(X_train, y_train_cat, batch_size=32, shuffle=True)
val_generator = val_datagen.flow(X_val, y_val_cat, batch_size=32, shuffle=False)

In [ ]:
num_classes = len(np.unique(y_train))                               # infer how many classes you have
input_shape = X_train.shape[1:]

In [ ]:
input_shape

Custom_CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=-1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(rate=0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(learning_rate= 0.001), loss='categorical_crossentropy'
, metrics= ['accuracy'])

In [ ]:
callbacks = [
    EarlyStopping(
        patience=7,
        restore_best_weights=True,
        monitor='val_accuracy'
    ),
    ReduceLROnPlateau(
        patience=3,
        factor=0.5,
        min_lr=1e-5,
        monitor='val_loss'
    ),
    ModelCheckpoint(
        filepath='best_gtsrb_cnn.keras',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [ ]:
history = model.fit(
    train_generator,                                                       # augmented training data
    validation_data=val_generator,                                         # clean validation data
    epochs=10,                                                       # cap epochs; early stopping will trigger
    callbacks=callbacks,
    verbose=1
)

In [ ]:
val_loss, val_acc = model.evaluate(val_generator, verbose=0)
print(f"Validation accuracy: {val_acc:.4f}")

In [ ]:
# get true labels from the generator (one-hot encoded -> convert to class indices)
y_val_true = np.argmax(val_generator.y, axis=1)


val_steps = len(val_generator)   # number of batches in the generator


# predictions
y_val_pred_probs = model.predict(val_generator, steps=val_steps, verbose=1)
y_val_pred = np.argmax(y_val_pred_probs, axis=1)

# confusion matrix
cm = confusion_matrix(y_val_true, y_val_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, cmap="Blues", fmt='g')
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# classification report
print(classification_report(y_val_true, y_val_pred))

Pre_Trained model

In [ ]:
# Load base model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
)
base_model.trainable = False  # Freeze base layers

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(43, activation='softmax')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=predictions)

mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train MobileNetV2
checkpoint_mobilenet = ModelCheckpoint('best_mobilenet.keras', save_best_only=True, monitor='val_accuracy', mode='max')

history_mobilenet = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint_mobilenet]
)

In [ ]:
from datetime import datetime
model.save(f"/kaggle/working/best_mobilenet_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras")

Compare between Custom CNN and Pre-trained Model

In [ ]:
# Predictions for Custom CNN
custom_preds = model.predict(val_generator)
custom_preds_classes = np.argmax(custom_preds, axis=1)
true_classes = np.argmax(y_val_cat, axis=1)

# Predictions for MobileNet
mobilenet_preds = mobilenet_model.predict(val_generator)
mobilenet_preds_classes = np.argmax(mobilenet_preds, axis=1)

# Confusion matrix for Custom CNN
cm_cnn = confusion_matrix(true_classes, custom_preds_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_cnn, annot=False, cmap="Blues")
plt.title("Custom CNN Confusion Matrix")
plt.show()
# Confusion matrix for MobileNet
cm_mobile = confusion_matrix(true_classes, mobilenet_preds_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_mobile, annot=False, cmap="Greens")
plt.title("MobileNet Confusion Matrix")
plt.show()

# Classification report
print("Custom CNN Report:\n", classification_report(true_classes, custom_preds_classes))
print("MobileNet Report:\n", classification_report(true_classes, mobilenet_preds_classes))

GTSRB Test Evaluation Script

In [ ]:
from sklearn.metrics import accuracy_score
# Paths - make sure these are correct
PATH = "/content/drive/MyDrive/Traffic_Sign_Recognition _Description"  # Adjust this path
data_dir = "/content/drive/MyDrive/Traffic_Sign_Recognition _Description"  # Adjust this path

test = pd.read_csv(os.path.join(PATH, 'Test.csv'))

labels = test["ClassId"].values
imgs = test["Path"].values

print(f"Total images in CSV: {len(imgs)}")

# Preprocess test images (32x32 for both models)
data = []
successful_indices = []  # To keep track of which images loaded successfully
successful_labels = []   # Corresponding labels for successful images

for i, img_path in enumerate(imgs):
   try:
        # Construct full path
        full_img_path = os.path.join(data_dir, img_path)

        # Check if file exists
        if not os.path.exists(full_img_path):
            print(f"File not found: {full_img_path}")
            continue

        image = cv2.imread(full_img_path)

        # Check if image was loaded correctly
        if image is None:
            print(f"Failed to load image: {img_path}")
            continue

        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((32, 32))
        data.append(np.array(resize_image))
        successful_indices.append(i)
        successful_labels.append(labels[i])

    except Exception as e:
        print(f"Error in {img_path}: {str(e)}")

# Check if any images were loaded
if len(data) == 0:
    print("❌ No images were loaded successfully! Check your file paths.")
    print("Available files in data_dir:")
    try:
        files = os.listdir(data_dir)
        print(f"First 10 files: {files[:10]}")
    except:
        print("Cannot list directory - path might be wrong")
else:
    X_test = np.array(data)
    X_test = X_test / 255.0   # normalization

    # ✅ Keep the data as images (32, 32, 3)
    print(f"X_test shape: {X_test.shape}")
     print(f"Successfully loaded {X_test.shape[0]} out of {len(imgs)} images")

    # Filter labels to only include successful images
    filtered_labels = np.array(successful_labels)

    # Predictions with both models using the same preprocessed data
    print("Making predictions with Custom Model...")
    y_pred_probs_custom = model.predict(X_test, batch_size=32, verbose=1)
    pred_custom = np.argmax(y_pred_probs_custom, axis=1)

    print("Making predictions with MobileNet Model...")
    y_pred_probs_mobilenet = mobilenet_model.predict(X_test, batch_size=32, verbose=1)
    pred_mobilenet = np.argmax(y_pred_probs_mobilenet, axis=1)

    # Accuracy for both models
    custom_accuracy = accuracy_score(filtered_labels, pred_custom) * 100
    mobilenet_accuracy = accuracy_score(filtered_labels, pred_mobilenet) * 100

    print('Custom Model Test Accuracy: ', custom_accuracy)
    print('MobileNet Test Accuracy: ', mobilenet_accuracy)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Select 10 random indices - CORRECTED LINE
random_indices = np.random.choice(range(len(X_test)), 10, replace=False)

# Create a figure to display the results
plt.figure(figsize=(20, 12))

for i, idx in enumerate(random_indices):
    # Get the image and true label
    image = X_test[idx]
    true_label = labels[idx]

    # Get predictions from both models
    custom_pred = pred_custom[idx]
    mobilenet_pred = pred_mobilenet[idx]

    # Check if predictions are correct
    custom_correct = (custom_pred == true_label)
    mobilenet_correct = (mobilenet_pred == true_label)

    # Create subplot
    plt.subplot(2, 5, i+1)

    # Display the image
    plt.imshow(image)

    # Set title with prediction results
    title = f'True: {true_label}\n'
    title += f'Custom: {custom_pred} '
    title += '✓' if custom_correct else '✗'
    title += f'\nMobileNet: {mobilenet_pred} '
    title += '✓' if mobilenet_correct else '✗'

    # Color code: green for correct, red for wrong
    color = 'green' if (custom_correct and mobilenet_correct) else 'red'

    plt.title(title, color=color, fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.suptitle('Predictions on 10 Random Images\n(✓ = Correct, ✗ = Wrong)', fontsize=16, y=1.02)
plt.show()

# Print detailed results
print("Detailed Results for 10 Random Images:")
print("=" * 50)
for i, idx in enumerate(random_indices):
    true_label = labels[idx]
    custom_pred = pred_custom[idx]
    mobilenet_pred = pred_mobilenet[idx]

    custom_status = "CORRECT" if custom_pred == true_label else "WRONG"
    mobilenet_status = "CORRECT" if mobilenet_pred == true_label else "WRONG"

    print(f"Image {i+1}: True Label = {true_label}")
    print(f"  Custom Model: {custom_pred} ({custom_status})")
    print(f"  MobileNet: {mobilenet_pred} ({mobilenet_status})")
    print("-" * 30)
    # Calculate agreement between models
agreement_count = sum(1 for i in range(len(labels)) if pred_custom[i] == pred_mobilenet[i])
agreement_percentage = (agreement_count / len(labels)) * 100

print(f"\nModel Agreement: {agreement_percentage:.2f}% of predictions are the same")
print(f"Custom Model Accuracy: {custom_accuracy:.2f}%")
print(f"MobileNet Accuracy: {mobilenet_accuracy:.2f}%")